In [21]:
import yake
import sys
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import requests
import time
from IPy import IP
from collections import defaultdict
import itertools  #used to slice a dictionary

import json
import pickle

In [22]:
#Functions used for extraction of text from the url provided

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def text_from_title(body):
    soup = BeautifulSoup(body, 'html.parser')
    title_text = soup.find('title').text
    return title_text

In [23]:
skip_domains = [
    'www.mail.google.com',
    'www.mail.yahoo.com',
    'www.account.google.com',
    'www.facebook.com',  
    'www.docs.google.com'
]

def isIP(str):
    try:
        IP(str)
    except ValueError:
        return False
    return True

def generate_clean_urls(url_file, clean_url_file):
    url_file = open(url_file, 'r')
    clean_urls = open(clean_url_file, 'w')
    for url in url_file.readlines():
        url = url.strip()
        spltAr = url.split("://");
        protocol = spltAr[0]
        if protocol not in ['http', 'https']:
            continue
        i = (0,1)[len(spltAr)>1];
        dm = spltAr[i].split("?")[0].split('/')[0].split(':')[0].lower();
        if dm[0:4] != 'www.':
            # check if domain is ip address
            if isIP(dm):
                continue
            dm = 'www.' + dm
        # domain checks
        if dm in skip_domains:
            continue
        clean_urls.write(url+'\n')
    url_file.close()
    clean_urls.close()

In [24]:
def keywordExtractor(url):
    url = url.strip()
    headers={'User-Agent': 'Mozilla/5.0'}
    resp = requests.get(url, headers=headers, timeout=3) 
    if resp.status_code >= 300:
        return
    html = resp.text
    text = text_from_html(html)
    title_text = text_from_title(html).lower()
    max_ngram_size = 1
    simple_kwextractor = yake.KeywordExtractor(lan="en", n = max_ngram_size, dedupLim=0.9, dedupFunc='seqm', windowsSize=1, top=20, features=None)
    keywords = simple_kwextractor.extract_keywords(text)
    title_keywords = simple_kwextractor.extract_keywords(title_text)
    title_words = dict([(t[1], t[0]) for t in title_keywords])  #to swap the term and its score to get {term:score}
    title_words = title_words.keys()
    
    keywords2 = []
    for kw in keywords:
        #since the order of term and score is reversed
        if kw[1] in title_words:
            kw2 = (kw[0]/4, kw[1])
        else:
            kw2 = kw
        keywords2.append(kw2)   #should be inside for loop so that all words are appended
    keywords2 = dict([(t[1], t[0]) for t in keywords2]) #to make a dictionary(as given in description) and also to reverse the order to {term:score}
    return keywords2

In [26]:
# load new history created since last update

new_history = 'history.txt'
cleaned_history = 'history_clean.txt'

generate_clean_urls(new_history, cleaned_history)

In [34]:
# keywords database structure

keywords_db = {
    'python' : {
        'title'    : 1.0, 
        'body'     : 1.0, 
        'bookmark' : 1.0
    },
}

#load keywords database from json file

filename = "keywords_db.data"

# try:
#     with open(filename, 'rb') as f:
#         keywords_db = pickle.load(f)
# except:
#     keywords_db = {}


In [35]:
# new keywords that we need to add as nodes/add edges for 
# in our clustering graph.

new_keywords_to_cluster = [['a', 'b'], ['c']]

In [36]:
def process_new_history(history_url_file):
    
    history = open(history_url_file, 'r')
    
    for line in history.readlines():
        # add code to get time spent on this webpage, scroll, etc.
        # should be present in history file
        try:
            # taking 5 most significant keywords
            keywords_list = keywordExtractor(line).items()
            keywords_list = sorted(keywords_list, key=lambda key: key[1])
            keywords_list = keywords_list[:5]
            
            # add keywords to be clustered
            new_keywords_to_cluster.append(
                [word for (word, value) in keywords_list])
            
            #for kw in keywords[:5]:
                # code to update score of keyword
                
        except:
            continue
            
    
    history.close()
    

In [38]:
# save keywords to cluster
with open('to_cluster.data', 'wb') as outfile:
    pickle.dump(new_keywords_to_cluster, outfile)

In [39]:
# save keywords database
with open('keyword_db.data', 'wb') as outfile:
    pickle.dump(keywords_db, outfile)